<a href="https://colab.research.google.com/github/ashikshafi08/AICrowd_Competition_Files/blob/main/TensorFlow/Rover_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Mar 27 01:44:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Rover Classification 
Someone has mishandled the labeling of two Mars rover projects — Curiosity and Perseverance — we must classify them correctly.

The given dataset contains images of two different rovers i.e. Curiosity and Perseverance of size 265x256 in jpg format. The images in **train.zip** and **val.zip**  have their labels i.e. which rover it is in **train.csv** and **val.csv**. The labels for the images in test.zip needs to be predicted.

### Evaluation Criteria
During evaluation F1 score is used as Primary Score and Accuracy Score as Secondary Score will be used to test the efficiency of the model.





In [2]:
# Downloading the data and putting them inside a folder
!pip install aicrowd-cli 
API_KEY = '166125ed5356c39ba63c026cf8cda906'
!aicrowd login --api-key $API_KEY

     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 163kB 7.8MB/s 
     |████████████████████████████████| 204kB 13.2MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
API Key valid
Saved API Key successfully!


In [3]:
# Downloading the datasets
!aicrowd dataset download --challenge rover-classification -j 3

sample_submission.csv:   0% 0.00/164k [00:00<?, ?B/s]
train.csv:   0% 0.00/689k [00:00<?, ?B/s]
sample_submission.csv: 100% 164k/164k [00:00<00:00, 1.86MB/s]
train.csv: 100% 689k/689k [00:00<00:00, 4.72MB/s]
test.zip:   0% 0.00/66.5M [00:00<?, ?B/s]
val.csv: 100% 65.0k/65.0k [00:00<00:00, 1.48MB/s]

train.zip:   0% 0.00/266M [00:00<?, ?B/s]

val.zip:   0% 0.00/26.5M [00:00<?, ?B/s]

val.zip: 100% 26.5M/26.5M [00:01<00:00, 15.5MB/s]

test.zip:  50% 33.6M/66.5M [00:05<00:05, 6.39MB/s]
train.zip:  25% 67.1M/266M [00:05<00:18, 11.0MB/s]
test.zip: 100% 66.5M/66.5M [00:08<00:00, 7.28MB/s]
test.zip: 100% 66.5M/66.5M [00:08<00:00, 8.01MB/s]
train.zip: 100% 266M/266M [00:18<00:00, 14.0MB/s]


In [ ]:
# Creating folders
!rm -rf data
!mkdir data

!unzip train.zip  -d data/train
!unzip val.zip -d data/val
!unzip test.zip  -d data/test

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

In [5]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-03-27 01:45:26--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9304 (9.1K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.09K  --.-KB/s    in 0s      

2021-03-27 01:45:26 (119 MB/s) - ‘helper_functions.py’ saved [9304/9304]



In [6]:
# Importing the needed packages 
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from helper_functions import plot_loss_curves , make_confusion_matrix , compare_historys , walk_through_dir 

In [7]:
# Looking inside the train directory
walk_through_dir('data/train/')
walk_through_dir('data/test/')
walk_through_dir('data/val/')


There are 0 directories and 40000 images in 'data/train/'.
There are 0 directories and 10000 images in 'data/test/'.
There are 0 directories and 4000 images in 'data/val/'.


In [8]:
# Let's get our csv files which will help us to label 
train_df = pd.read_csv('data/train.csv')
val_df = pd.read_csv('data/val.csv')

In [9]:
# Checking what's in the train_df
train_df.head()

,ImageID,label
0,0,curiosity
1,1,curiosity
2,2,curiosity
3,3,perseverance
4,4,curiosity


In [18]:
# How many unique classe are there? (and great there is not class imbalance)
train_df['label'].value_counts()

perseverance    20048
curiosity       19952
Name: label, dtype: int64

In [10]:
# Getting the train, test and val dir 
train_dir = 'data/train/'
test_dir = 'data/test/'
val_dir = 'data/val/'

In [11]:
# Will look how the filenames look like 
import os 
print(os.listdir(train_dir)[:10])

['13794.jpg', '1058.jpg', '37500.jpg', '8855.jpg', '10105.jpg', '23535.jpg', '25702.jpg', '23006.jpg', '2752.jpg', '6572.jpg']


In our dataframe we gotta add the extension `jpg` at the end to download the images. 

In [12]:
# Creating a function for making the action
def append_ext(fn):
    return f'{fn}.jpg'

# Now applying them 
train_df['ImageID'] = train_df['ImageID'].apply(append_ext)
val_df['ImageID'] = val_df['ImageID'].apply(append_ext)

In [13]:
# Looking into to make sure how the changes are 
train_df['ImageID'][0]

'0.jpg'

In [15]:
# Creating Data generator for our data 
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1 / 255.)

In [20]:
# Now it's time to import our data into the generator 

train_data = train_datagen.flow_from_dataframe(dataframe= train_df , 
                                               directory = train_dir , 
                                               x_col = 'ImageID' , 
                                               y_col = 'label' , 
                                               target_size = (224 , 224) , 
                                               class_mode = 'binary')

val_data = val_datagen.flow_from_dataframe(dataframe = val_df , 
                                           directory = val_dir , 
                                           x_col = 'ImageID' , 
                                           y_col = 'label' , 
                                           target_size = (224 , 224) , 
                                           class_mode = 'binary')

Found 40000 validated image filenames belonging to 2 classes.
Found 4000 validated image filenames belonging to 2 classes.


But here is the tricky part, the data augmentation will run on CPU if I am gonna use `ImageDataGenerator` which will slow up the entire process. By using `image_dataset_from_directory` at further we could create data augmentation as an layer and make the augmentation process run on GPU. 

Ima try If I can do that by using `ImageDataGenerator`, still not sure how it's gonna go. Experiment! 

In [29]:
from tensorflow.keras.layers.experimental import preprocessing

data_augmentation_layer = tf.keras.Sequential([
  preprocessing.RandomWidth(0.2), 
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomFlip() , 
  preprocessing.RandomHeight(0.2)
])

Let's build a baseline and see how it goes! Will build: 
- A model which has no data augmentation, just from a ImageDataGenerator with Transfer Learning (EfficientNetB0) and run for few epochs. 
- A experimentation model which does have augmentation like a layer, but unlike generator gotta add this inside the model to make it run on GPU. 
- If the above doesn't work, gotta add augmentation in the Datagenerator, and train a model.